In [1]:
import warnings  

import os
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import Subset
from torch.nn.functional import softmax

from efficientnet_pytorch import EfficientNet

from typing import Callable, List, Tuple, Dict
from pathlib import Path

from transformers import AdamW
from transformers import get_cosine_schedule_with_warmup

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from collections import defaultdict, OrderedDict
from tqdm.notebook import tqdm
from torchsummary import summary

from fastai2.layers import LabelSmoothingCrossEntropy
from transformers import get_cosine_schedule_with_warmup
from transformers import AdamW

import matplotlib
matplotlib.rcParams.update({'figure.figsize': (16, 12), 'font.size': 14})
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output

In [2]:
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", DeprecationWarning)
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
EXPERIMENT_NAME = "03_starter_code"

class ConfigExperiment:
    logdir = f"./logs/{EXPERIMENT_NAME}"
    save_dirname = EXPERIMENT_NAME
    submission_file = f"{EXPERIMENT_NAME}.csv"
    seed = 42
    batch_size = 32
    model_name = 'efficientnet-b0'
    size = 512
    num_workers = 20
    root_images = "../../../data/raw/plant-pathology-2020-fgvc7/images/"
    root = "../../../data/raw/plant-pathology-2020-fgvc7/"
    num_classes = 4
    patience = 10
    early_stopping_delta = 1e-4
    num_epochs = 200
    lr = 8e-4
    class_names = ["healthy", "multiple_diseases", "rust", "scab"]
    is_fp16_used = False
    
    
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
    
config = ConfigExperiment()
set_seed(config.seed)
config.size = EfficientNet.get_image_size(config.model_name)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = "cuda" if torch.cuda.is_available() else "cpu"

try:
    # Create target Directory
    os.mkdir(config.save_dirname)
    print("Directory " , config.save_dirname ,  " Created ") 
except FileExistsError:
    print("Directory " , config.save_dirname ,  " already exists")

Directory  03_starter_code  already exists


In [4]:
class PlantDataset(Dataset):
    
    def __init__(self, df, config, transforms=None):
    
        self.df = df
        self.images_dir = config.root_images
        self.class_names = config.class_names
        self.transforms=transforms
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        image_src = self.images_dir + self.df.iloc[idx]['image_id'] + '.jpg'
        image = cv2.imread(image_src, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        labels = self.df.iloc[idx][self.class_names].values.astype(np.int8)
        label = torch.argmax(torch.from_numpy(labels))
        
        if self.transforms:
            transformed = self.transforms(image=image)
            image = transformed['image']

        return image, label

In [5]:
def pre_transforms(image_size=224):
    # Convert the image to a square of size image_size x image_size
    # (keeping aspect ratio)
    result = [
        A.LongestMaxSize(max_size=image_size),
        A.PadIfNeeded(image_size, image_size, border_mode=0)
    ]
    
    return result

def hard_transforms():
    result = [
        # Random shifts, stretches and turns with a 50% probability
#         A.RandomResizedCrop(height=config.size, width=config.size, p=1.0),
        A.OneOf([
            A.Rotate(limit=90, p=1),
            A.HorizontalFlip(p=1),
            A.VerticalFlip(p=1),
            A.OpticalDistortion(p=1),
            A.ShiftScaleRotate(rotate_limit=90, shift_limit=0.2, scale_limit=0.1, p=1)
        ], p=0.8),
        
        # Pixels
        A.OneOf([
            A.IAAEmboss(p=1.0),
            A.IAASharpen(p=1.0),
            A.Blur(p=1.0),
        ], p=0.5),

        # Affine
        A.OneOf([
            A.RandomContrast(limit=0.2, p=1),
            A.ElasticTransform(p=1),
            A.IAAPiecewiseAffine(p=1)
        ], p=0.5),
    ]
    
    return result

def post_transforms():
    # we use ImageNet image normalization
    # and convert it to torch.Tensor
    return [A.Normalize(p=1.0), ToTensorV2(p=1.0),]

def compose(transforms_to_compose):
    # combine all augmentations into one single pipeline
    result = A.Compose([item for sublist in transforms_to_compose for item in sublist])
    return result

In [6]:
train_transforms = compose([
    pre_transforms(config.size),
    hard_transforms(), 
    post_transforms()
])
valid_transforms = compose([
    pre_transforms(config.size), 
    post_transforms()
])

show_transforms = compose([
    pre_transforms(config.size),
    hard_transforms()
])

In [7]:
def get_model(model_name: str, num_classes: int, pretrained: str = "imagenet") -> EfficientNet:
    model = EfficientNet.from_pretrained(model_name)
    for param in model.parameters():
        param.requires_grad = False
    num_ftrs = model._fc.in_features
    model._fc = nn.Sequential(nn.Linear(num_ftrs, num_classes, bias = True))
    return model

In [8]:
class Trainer:
    def __init__(self, model, train_dataloader: DataLoader, valid_dataloader: DataLoader, 
                 criterion, 
                 optimizer, 
                 scheduler, 
                 device, 
                 config: ConfigExperiment,
                 prefix: str=""
                ):
        self.model = model
        self.train_dataloader = train_dataloader
        self.valid_dataloader = valid_dataloader
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.device = device
        self.config = config
        self.prefix = prefix
        self.chekpoint_loss = ""
        self.chekpoint_auc = ""
        self.train_metrics = {
            'avg_loss': [],
            'auc/_mean': [],
            'auc/healthy': [],
            'auc/multiple_diseases': [],
            'auc/rust': [],
            'auc/scab': [],
        }
        self.valid_metrics = {
            'avg_loss': [],
            'auc/_mean': [],
            'auc/healthy': [],
            'auc/multiple_diseases': [],
            'auc/rust': [],
            'auc/scab': [],
        }
        self.counter = 0
        self.delta = config.early_stopping_delta
      
    def run(self):
        self.model.to(device)
        best_valid_loss = float('inf')
        best_valid_auc_mean = 0

        try:
            for i_epoch in tqdm(range(self.config.num_epochs), desc='Epochs', total=config.num_epochs, position=1, leave=True):
                start_time = time.time()

                train_loss, train_outputs, train_targets = self._train()
                valid_loss, valid_outputs, valid_targets = self._evaluate()
                    
                self.train_metrics["avg_loss"].append(train_loss)
                self.train_metrics["auc/_mean"].append(self.comp_metric(train_outputs, train_targets))
                self.train_metrics["auc/healthy"].append(self.healthy_roc_auc(train_outputs, train_targets))
                self.train_metrics["auc/multiple_diseases"].append(self.multiple_diseases_roc_auc(train_outputs, train_targets))
                self.train_metrics["auc/rust"].append(self.rust_roc_auc(train_outputs, train_targets))
                self.train_metrics["auc/scab"].append(self.scab_roc_auc(train_outputs, train_targets))
                
                self.valid_metrics["avg_loss"].append(valid_loss)
                self.valid_metrics["auc/_mean"].append(self.comp_metric(valid_outputs, valid_targets))
                self.valid_metrics["auc/healthy"].append(self.healthy_roc_auc(valid_outputs, valid_targets))
                self.valid_metrics["auc/multiple_diseases"].append(self.multiple_diseases_roc_auc(valid_outputs, valid_targets))
                self.valid_metrics["auc/rust"].append(self.rust_roc_auc(valid_outputs, valid_targets))
                self.valid_metrics["auc/scab"].append(self.scab_roc_auc(valid_outputs, valid_targets))
                
                end_time = time.time()
                epoch_mins, epoch_secs = self._epoch_time(start_time, end_time)
                self.print_progress(i_epoch, epoch_mins, epoch_secs)
                
                self.scheduler.step(self.valid_metrics["auc/_mean"][-1])
                
                if valid_loss < best_valid_loss:
                    self.counter = 0
                    best_valid_loss = valid_loss
                    chekpoint = f"{config.save_dirname}/{self.prefix}best_model_loss={best_valid_loss}.pth"
                    torch.save(model.state_dict(), chekpoint)
                    if self.chekpoint_loss:
                        os.remove(self.chekpoint_loss)
                    self.chekpoint_loss = chekpoint
                else:
                    self.counter += 1
                    
                if self.valid_metrics["auc/_mean"][-1] > best_valid_auc_mean:
                    best_valid_auc_mean = self.valid_metrics["auc/_mean"][-1]
                    chekpoint = f"{config.save_dirname}/{self.prefix}best_model_auc_mean={best_valid_auc_mean}.pth"
                    torch.save(model.state_dict(), chekpoint)
                    if self.chekpoint_auc:
                        os.remove(self.chekpoint_auc)
                    self.chekpoint_auc = chekpoint
                    
                if self.counter > self.config.patience:
                    print("EarlyStopping")
                    break
        except KeyboardInterrupt: 
            pass
        
        return self.train_metrics, self.valid_metrics, self.chekpoint_loss, self.chekpoint_auc
        
    def _train(self):
        model.train()
        epoch_loss = 0
        epoch_output = None
        epoch_target = None
        for i, (images, labels) in tqdm(enumerate(self.train_dataloader), desc='Train', total=len(self.train_dataloader), position=2, leave=True):
            loss_iten, outputs = self._train_process(images, labels)
            epoch_loss += loss_iten              

            if epoch_output is None:
                epoch_output = outputs.cpu().data
            else:
                epoch_output = torch.cat((epoch_output, outputs.cpu().data))

            if epoch_target is None:
                epoch_target = labels.cpu().data
            else:
                epoch_target = torch.cat((epoch_target, labels.cpu().data))
            
        return epoch_loss / len(self.train_dataloader), epoch_output, epoch_target
    
    def _train_process(self, images, labels):
        images = images.to(self.device)
        labels = labels.to(self.device)
        self.optimizer.zero_grad()
        outputs = self.model(images)
        loss = self.criterion(outputs, labels)
        loss.backward()
        self.optimizer.step()
        return loss.item(), outputs
            
    def _evaluate(self):
        model.eval()
        epoch_loss = 0
        epoch_output = None
        epoch_target = None
        with torch.no_grad():
            for i, (images, labels) in enumerate(self.valid_dataloader):
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                epoch_loss += loss.item()
                
                if epoch_output is None:
                    epoch_output = outputs.cpu().data
                else:
                    epoch_output = torch.cat((epoch_output, outputs.cpu().data))

                if epoch_target is None:
                    epoch_target = labels.cpu().data
                else:
                    epoch_target = torch.cat((epoch_target, labels.cpu().data))

        return epoch_loss / len(self.valid_dataloader), epoch_output, epoch_target
    
#     def _evaluate_tta(self):
#         n_samples = len(dataset)
#         y_probas_tta = np.zeros((n_samples, config.num_classes, tta_count), dtype=np.float32)

#         model.eval()
#         model.to(device)
#         with torch.no_grad():
#             for i_epoch in tqdm(range(tta_count), desc='TTA', total=tta_count, position=1, leave=True):
#                 for i, (images, idx) in enumerate(dataloader):
#                     images = images.to(device)
#                     outputs = model(images)
#                     y_pred = F.softmax(outputs, dim=1)

#                     tta_index = i_epoch - 1
#                     start_index = (i % len(dataloader)) * config.batch_size
#                     end_index = min(start_index + config.batch_size, n_samples)
#                     batch_y_probas = y_pred.detach().cpu().numpy()                
#                     y_probas_tta[start_index:end_index, :, tta_index] = batch_y_probas

#         return y_probas_tta

#     def show_history(self):
#         fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 8))
#         clear_output(True)
#         ax[0].plot(history, label='train loss')
#         ax[0].set_xlabel('Batch')
#         ax[0].set_title('Train loss')
#         if train_history is not None:
#             ax[1].plot(train_history, label='general train history')
#             ax[1].set_xlabel('Epoch')
#         if valid_history is not None:
#             ax[1].plot(valid_history, label='general valid history')
#         plt.legend()
#         plt.show()

    
    def _epoch_time(self, start_time, end_time):
        elapsed_time = end_time - start_time
        elapsed_mins = int(elapsed_time / 60)
        elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
        return elapsed_mins, elapsed_secs

    def print_progress(self, i_epoch, epoch_mins, epoch_secs):
        i_epoch = i_epoch + 1
        print(f"Epoch: {i_epoch:02} | Time: {epoch_mins}m {epoch_secs}s")
        print("Training Results - Average Loss: {:.4f} | auc/_mean: {:.4f} | auc/healthy: {:.4f} | auc/multiple_diseases: {:.4f}, auc/rust: {:.4f}, auc/scab: {:.4f}"
            .format(
                self.train_metrics['avg_loss'][-1], 
                self.train_metrics['auc/_mean'][-1],
                self.train_metrics['auc/healthy'][-1],
                self.train_metrics['auc/multiple_diseases'][-1],
                self.train_metrics['auc/rust'][-1],
                self.train_metrics['auc/scab'][-1],
            ))
        print("Evaluating Results - Average Loss: {:.4f} | auc/_mean: {:.4f} | auc/healthy: {:.4f} | auc/multiple_diseases: {:.4f}, auc/rust: {:.4f}, auc/scab: {:.4f}"
            .format( 
                self.valid_metrics['avg_loss'][-1], 
                self.valid_metrics['auc/_mean'][-1],
                self.valid_metrics['auc/healthy'][-1],
                self.valid_metrics['auc/multiple_diseases'][-1],
                self.valid_metrics['auc/rust'][-1],
                self.valid_metrics['auc/scab'][-1],
            ))
        print()
        
    def comp_metric(self, preds, targs, labels=range(4)):
        preds = torch.sigmoid(preds)
        targs = torch.eye(4)[targs]
        return np.mean([roc_auc_score(targs[:,i], preds[:,i]) for i in labels])

    def healthy_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[0])

    def multiple_diseases_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[1])

    def rust_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[2])

    def scab_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[3])



In [9]:
train_df = pd.read_csv(config.root + 'train.csv')
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=config.seed)
maximas=np.argmax(np.array(train_df.drop('image_id',axis=1)),axis=1)
final_model_checkpoints = []
for i, (train_idx, valid_idx) in enumerate(skf.split(train_df, maximas)):
    # Create Datasets and Dataloaders
    fold_train_df = train_df.iloc[train_idx].reset_index()
    fold_valid_df = train_df.iloc[valid_idx].reset_index()
    train_dataset = PlantDataset(fold_train_df, config, train_transforms)
    valid_dataset = PlantDataset(fold_valid_df, config, valid_transforms)
    train_dataloader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=config.num_workers)
    valid_dataloader = DataLoader(valid_dataset, batch_size=config.batch_size, shuffle=False, num_workers=config.num_workers)
    
    # Model I
    model = get_model(config.model_name, config.num_classes)
    criterion = LabelSmoothingCrossEntropy()
    optimizer = AdamW(model.parameters(), lr=config.lr, weight_decay=1e-3)
    num_train_steps = int(len(train_dataset) / config.batch_size * config.num_epochs)
    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=len(train_dataset)/config.batch_size*5, num_training_steps=num_train_steps)
    # Train model I
    trainer = Trainer(model, train_dataloader, valid_dataloader, criterion, optimizer, scheduler, device, config, prefix=f"fold_{i}_I_")
    _, _, checpoint, _ = trainer.run();
    
    # Model II
    model = get_model(config.model_name, config.num_classes)
    model.load_state_dict(torch.load(checpoint))
    for param in model.parameters():
        param.requires_grad = True
    criterion = LabelSmoothingCrossEntropy()
    optimizer = AdamW(model.parameters(), lr=config.lr, weight_decay=1e-3)
    num_train_steps = int(len(train_dataset) / config.batch_size * config.num_epochs)
    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=len(train_dataset)/config.batch_size*5, num_training_steps=num_train_steps)
    # Train model II
    trainer = Trainer(model, train_dataloader, valid_dataloader, criterion, optimizer, scheduler, device, config, prefix=f"fold_{i}_II_")
    _, _, _, checkpoint = trainer.run();
    
    final_model_checkpoints.append(checkpoint)

Loaded pretrained weights for efficientnet-b0



Epoch: 01 | Time: 0m 13s
Training Results - Average Loss: 1.4009 | auc/_mean: 0.4818 | auc/healthy: 0.4918 | auc/multiple_diseases: 0.4848, auc/rust: 0.4421, auc/scab: 0.5085
Evaluating Results - Average Loss: 1.3784 | auc/_mean: 0.4616 | auc/healthy: 0.5021 | auc/multiple_diseases: 0.5789, auc/rust: 0.3012, auc/scab: 0.4640




Epoch: 02 | Time: 0m 13s
Training Results - Average Loss: 1.3945 | auc/_mean: 0.4824 | auc/healthy: 0.4603 | auc/multiple_diseases: 0.4542, auc/rust: 0.4884, auc/scab: 0.5266
Evaluating Results - Average Loss: 1.3824 | auc/_mean: 0.4508 | auc/healthy: 0.4482 | auc/multiple_diseases: 0.5581, auc/rust: 0.3265, auc/scab: 0.4704




Epoch: 03 | Time: 0m 13s
Training Results - Average Loss: 1.3967 | auc/_mean: 0.4847 | auc/healthy: 0.4683 | auc/multiple_diseases: 0.4925, auc/rust: 0.4574, auc/scab: 0.5204
Evaluating Results - Average Loss: 1.3837 | auc/_mean: 0.4510 | auc/healthy: 0.4084 | auc/multiple_diseases: 0.5141, auc/rust: 0.3730, auc/scab: 0.5087




Epoch: 04 | Time: 0m 13s
Training Results - Average Loss: 1.3941 | auc/_mean: 0.4836 | auc/healthy: 0.4791 | auc/multiple_diseases: 0.4998, auc/rust: 0.4530, auc/scab: 0.5023
Evaluating Results - Average Loss: 1.3786 | auc/_mean: 0.4643 | auc/healthy: 0.4162 | auc/multiple_diseases: 0.4997, auc/rust: 0.4031, auc/scab: 0.5381



...


Epoch: 196 | Time: 0m 20s
Training Results - Average Loss: 0.5158 | auc/_mean: 0.9638 | auc/healthy: 0.9931 | auc/multiple_diseases: 0.8745, auc/rust: 0.9935, auc/scab: 0.9941
Evaluating Results - Average Loss: 0.5088 | auc/_mean: 0.9470 | auc/healthy: 0.9931 | auc/multiple_diseases: 0.8126, auc/rust: 0.9972, auc/scab: 0.9850




Epoch: 197 | Time: 0m 20s
Training Results - Average Loss: 0.5285 | auc/_mean: 0.9575 | auc/healthy: 0.9913 | auc/multiple_diseases: 0.8531, auc/rust: 0.9927, auc/scab: 0.9931
Evaluating Results - Average Loss: 0.5084 | auc/_mean: 0.9477 | auc/healthy: 0.9932 | auc/multiple_diseases: 0.8155, auc/rust: 0.9973, auc/scab: 0.9850




Epoch: 198 | Time: 0m 20s
Training Results - Average Loss: 0.5016 | auc/_mean: 0.9678 | auc/healthy: 0.9950 | auc/multiple_diseases: 0.8865, auc/rust: 0.9958, auc/scab: 0.9937
Evaluating Results - Average Loss: 0.5079 | auc/_mean: 0.9482 | auc/healthy: 0.9931 | auc/multiple_diseases: 0.8176, auc/rust: 0.9972, auc/scab: 0.9850




Epoch: 199 | Time: 0m 20s
Training Results - Average Loss: 0.5222 | auc/_mean: 0.9658 | auc/healthy: 0.9929 | auc/multiple_diseases: 0.8848, auc/rust: 0.9938, auc/scab: 0.9916
Evaluating Results - Average Loss: 0.5086 | auc/_mean: 0.9482 | auc/healthy: 0.9932 | auc/multiple_diseases: 0.8172, auc/rust: 0.9972, auc/scab: 0.9850




Epoch: 200 | Time: 0m 20s
Training Results - Average Loss: 0.5135 | auc/_mean: 0.9590 | auc/healthy: 0.9946 | auc/multiple_diseases: 0.8568, auc/rust: 0.9909, auc/scab: 0.9937
Evaluating Results - Average Loss: 0.5076 | auc/_mean: 0.9488 | auc/healthy: 0.9932 | auc/multiple_diseases: 0.8198, auc/rust: 0.9972, auc/scab: 0.9850




In [10]:
final_model_checkpoints

['03_starter_code/fold_0_II_best_model_auc_mean=0.9714526127751457.pth',
 '03_starter_code/fold_1_II_best_model_auc_mean=0.9664926966332525.pth',
 '03_starter_code/fold_2_II_best_model_auc_mean=0.9271799007470296.pth',
 '03_starter_code/fold_3_II_best_model_auc_mean=0.950173945301858.pth',
 '03_starter_code/fold_4_II_best_model_auc_mean=0.9487899259301332.pth']

In [11]:
class PlantDatasetTest(Dataset):
    
    def __init__(self, df, config, transforms=None):
    
        self.df = df
        self.images_dir = config.root_images
        self.class_names = config.class_names
        self.transforms=transforms
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        image_src = self.images_dir + self.df.iloc[idx]['image_id'] + '.jpg'
        image = cv2.imread(image_src, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms:
            transformed = self.transforms(image=image)
            image = transformed['image']

        return image, idx

test_df = pd.read_csv(config.root + 'test.csv')
test_dataset = PlantDatasetTest(test_df, config, valid_transforms)
test_dataloader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False, num_workers=config.num_workers)

In [12]:
def predict_kfold(model_list, dataloader: DataLoader, device, dataset: Dataset, config: ConfigExperiment):
    n_samples = len(dataset)
    y_probas = np.zeros((n_samples, config.num_classes, len(model_list)), dtype=np.float32)
    for i_epoch, checkpoint in enumerate(tqdm(model_list, desc='k-fold', total=len(model_list), position=1, leave=True)):
        model = get_model(config.model_name, config.num_classes)
        model.load_state_dict(torch.load(checpoint))
        model.eval()
        model.to(device)
        with torch.no_grad():
            for i, (images, idx) in enumerate(dataloader):
                images = images.to(device)
                outputs = model(images)
                y_pred = F.softmax(outputs, dim=1)

                start_index = (i % len(dataloader)) * config.batch_size
                end_index = min(start_index + config.batch_size, n_samples)
                batch_y_probas = y_pred.detach().cpu().numpy()                
                y_probas[start_index:end_index, :, i_epoch-1] = batch_y_probas

    return y_probas

In [13]:
result = predict_kfold(final_model_checkpoints, test_dataloader, device, test_dataset, config=config)

Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0



In [14]:
y_probas = np.mean(result, axis=-1)

In [15]:
y_probas[:10]

array([[0.2269903 , 0.05000266, 0.44586787, 0.27713922],
       [0.21503969, 0.0581282 , 0.45550594, 0.27132618],
       [0.16445768, 0.02720291, 0.34112188, 0.4672175 ],
       [0.43956456, 0.04287212, 0.25434613, 0.26321715],
       [0.19245121, 0.05534904, 0.5565583 , 0.19564149],
       [0.28747687, 0.04186786, 0.44192988, 0.22872548],
       [0.30338734, 0.04710042, 0.39271888, 0.25679338],
       [0.19254795, 0.12582962, 0.2768287 , 0.40479374],
       [0.27277586, 0.06236488, 0.39142218, 0.27343702],
       [0.27620986, 0.05535417, 0.47326845, 0.19516753]], dtype=float32)

In [16]:
test_df = pd.read_csv(config.root + 'test.csv')
test_df["healthy"] = 0
test_df["multiple_diseases"] = 0
test_df["rust"] = 0
test_df["scab"] = 0
test_df[['healthy', 'multiple_diseases', 'rust', 'scab']] = y_probas
test_df.to_csv(config.submission_file, index=False)
test_df.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0.226990,0.050003,0.445868,0.277139
1,Test_1,0.215040,0.058128,0.455506,0.271326
2,Test_2,0.164458,0.027203,0.341122,0.467218
3,Test_3,0.439565,0.042872,0.254346,0.263217
4,Test_4,0.192451,0.055349,0.556558,0.195641


In [17]:
config.submission_file

'03_starter_code.csv'

In [12]:
def predict_tta(model, dataloader: DataLoader, device, dataset: Dataset, tta_count: int, config: ConfigExperiment):
    n_samples = len(dataset)
    y_probas_tta = np.zeros((n_samples, config.num_classes, tta_count), dtype=np.float32)
    
    model.eval()
    model.to(device)
    with torch.no_grad():
        for i_epoch in tqdm(range(tta_count), desc='TTA', total=tta_count, position=1, leave=True):
            for i, (images, idx) in enumerate(dataloader):
                images = images.to(device)
                outputs = model(images)
                y_pred = F.softmax(outputs, dim=1)
                
                tta_index = i_epoch - 1
                start_index = (i % len(dataloader)) * config.batch_size
                end_index = min(start_index + config.batch_size, n_samples)
                batch_y_probas = y_pred.detach().cpu().numpy()                
                y_probas_tta[start_index:end_index, :, tta_index] = batch_y_probas

    return y_probas_tta

In [13]:
model = get_model(config.model_name, config.num_classes)
model.load_state_dict(torch.load(f"{config.save_dirname}/best_model_epoch=125_auc_mean=0.9759388963311403.pth"))

Loaded pretrained weights for efficientnet-b0


<All keys matched successfully>

In [14]:
result = predict(model, test_dataloader, device, test_dataset, tta_count=5, config=config)

In [15]:
y_probas = np.mean(result, axis=-1)

In [16]:
y_probas[:10]

array([[0.11525081, 0.03328629, 0.794872  , 0.05659093],
       [0.02689193, 0.08945929, 0.86251545, 0.02113327],
       [0.01418664, 0.03105108, 0.02221094, 0.9325514 ],
       [0.9593188 , 0.00964081, 0.01439708, 0.01664325],
       [0.01813011, 0.03230838, 0.92921793, 0.02034356],
       [0.91774046, 0.01537943, 0.03287008, 0.03401009],
       [0.9554141 , 0.01326885, 0.02219261, 0.00912439],
       [0.01802489, 0.20799896, 0.04579664, 0.72817945],
       [0.01979253, 0.5242635 , 0.03942884, 0.4165151 ],
       [0.03681117, 0.08224317, 0.8446248 , 0.03632084]], dtype=float32)

In [17]:
test_df = pd.read_csv(config.root + 'test.csv')
test_df["healthy"] = 0
test_df["multiple_diseases"] = 0
test_df["rust"] = 0
test_df["scab"] = 0
test_df[['healthy', 'multiple_diseases', 'rust', 'scab']] = y_probas
test_df.to_csv(config.submission_file, index=False)
test_df.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0.115251,0.033286,0.794872,0.056591
1,Test_1,0.026892,0.089459,0.862515,0.021133
2,Test_2,0.014187,0.031051,0.022211,0.932551
3,Test_3,0.959319,0.009641,0.014397,0.016643
4,Test_4,0.018130,0.032308,0.929218,0.020344


In [18]:
config.submission_file

'02_starter_code.csv'